In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"


In [1]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-4-h-small"

model_parameters = {
    "temperature": 0,
    "max_completion_tokens": 500,
    "repetition_penalty": 1.05,
    "stop": ["Observation:", "\nObservation:", " Observation:"],
}

llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=model_parameters,
)

Please enter your WATSONX_URL:  ········
Please enter your WATSONX_APIKEY:  ········
Please enter your WATSONX_PROJECT_ID:  ········


In [12]:
#definte mock tools for now as test
from langchain.tools import tool

@tool
def get_weather(destination: str, month: str) -> str:
    """Get weather for a destination and month."""
    return f"The weather in {destination} in {month} is cool and dry, highs ~15°C."

@tool
def get_flight_info(destination: str, month: str) -> str:
    """Get flight cost for a destination and month."""
    return f"Flights to {destination} in {month} average around $1200 USD round-trip."

@tool
def get_hotel_prices(destination: str, month: str) -> str:
    """Get hotel prices for a destination and month."""
    return f"Hotels in {destination} range from $100 to $300 per night in {month}."

tools = [get_weather, get_flight_info, get_hotel_prices]


In [13]:
from langchain_ibm import ChatWatsonx
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, AIMessage


In [14]:
tool_descriptions = "\n".join([f"- {t.name}: {t.description}" for t in tools])
tool_names = ", ".join([t.name for t in tools])

In [15]:
react_instructions = f"""
You are a helpful travel planning assistant that uses the ReAct (Reasoning + Acting) framework to answer travel-related questions.

Always reason step by step in a visible loop of **Thought → Action  → Observation**.
When finished, output the **Answer**.

Tool Usage
Available Tools to you:
{tool_descriptions}

You MUST use the following format to answer the user's travel related question, using the list of tools as needed:

Thought: <Explain thoughts and action to take>
Action: {{"name": "tool_name", "args": {{"destination": "...", "month":"..."}}}}
Observation: (filled later with the tool result)
(This loop may repeat multiple times, until you know the final answer)
Thought: I know the final answer.
Answer: <final answer to the user>

Example Format

User: I want to visit Paris in May. How much should that cost?
Thought: The user wants travel advice. I should check the hotel prices
Action: {{"name": "get_hotel_prices", "args": {{"destination": "Paris", "month": "May"}}
Observation: Hotel prices are between $250-500.
Thought: Next I need to check the price of flights
Action: {{"name": "get_flight_info, "args": {{"destination": "Paris", "month": "May"}}
Observation: Flight to Paris in May are $500.
Thought: I know the final answer.
Answer: Travel to Paris is good this time of year. Hotel prices are between $250-500 and flight tickets are around $500


Rules
* Always show each step begging with "Thought:" on it's own line BEFORE performing any actions.
* YOU MUST make only ONE Action at a time per loop
* NEVER propose multiple Actions together.
* After performing ONE Action, WAIT for the Observation before reasoning again
* DO NOT Write Observation yourself. Only the system will provide Observation after your Action
* Only use tools listed in "Available Tools", which are the following: {tool_names}
* Stop ONLY when you reach the final Answer
* NEVER include Final Answer in the same turn as an Action. If you just took an Action, wait for an Observation first.
* If you think no Action is needed, just output Answer, do not generate a JSON Action.
* Only use tools these tools: {tool_names}
"""



In [16]:
model_with_tools = llm

In [17]:
class AgentState(TypedDict):
    messages: Annotated[List[AIMessage], add_messages]

In [18]:
import json
from langchain_core.messages import AIMessage

def call_model(state: AgentState):
    """Single ReAct reasoning step without regex loops.
    Extracts everything between the first '{' and the last '}' after Action:.
    """
    messages = [("system", react_instructions), *state["messages"]]
    ai = llm.invoke(messages)
    output_text = (ai.content or "").strip()
    print(output_text)

    # --- extract Action JSON using first '{' and last '}' slice ---
    if "Action:" in output_text:
        # Grab everything after "Action:" and between outer braces
        raw_action = output_text.split("Action:", 1)[-1]
        start = raw_action.find("{")
        end = raw_action.rfind("}") + 1
        if start != -1 and end != -1:
            raw_action = raw_action[start:end]
        else:
            raw_action = None

        if raw_action:
            try:
                # Clean up single quotes if model uses them
                cleaned = raw_action.replace("'", '"')
                print("🧩 Parsed Action text:", cleaned)
                action = json.loads(cleaned)

                tool_name = action.get("name")
                args = action.get("args", {}) or {}
                tool = next((t for t in tools if t.name == tool_name), None)

                if tool:
                    obs = tool.invoke(args)
                    print(f"👁️ Observation: {obs}\n{'-'*70}")
                    # Feed observation back into conversation
                    return {"messages": [ai, ("user", f"Observation: {obs}")]}
                else:
                    print(f"⚠️ Unknown tool: {tool_name}")
            except Exception as e:
                print(f"⚠️ Failed to parse Action JSON: {e}")
                return {"messages": [ai]}  # fail-safe end

    # --- detect final Answer ---
    if "Answer:" in output_text:
        print("✅ Final Answer reached.\n")
        return {"messages": [ai]}

    # --- safety fallback ---
    print("🛑 No Action/Answer detected; ending to avoid loop.\n")
    return {"messages": [ai]}


In [19]:
def route_from_llm(state: AgentState):
    last = state["messages"][-1]
    if isinstance(last, AIMessage):
        txt = (last.content or "")
        # Final answer ends the run
        if "Answer:" in txt:
            return "end"
        # No Action and no Answer → end (prevents infinite loop)
        return "end"
    else:
        # A tuple like ("user", "Observation: ...") → continue to llm
        return "llm"

In [20]:
graph = StateGraph(AgentState)
graph.add_node("llm", call_model)
graph.add_edge(START, "llm")
graph.add_conditional_edges("llm", route_from_llm, {"llm": "llm", "end": END})
app = graph.compile()


In [21]:
inputs = {"messages": [("user", "I want to visit Tokyo next April. What should I expect about weather and costs?")]}
result = app.invoke(inputs)

Thought: The user wants to know about the weather and costs for a trip to Tokyo in April. I should check the weather first.
Action: {"name": "get_weather", "args": {"destination": "Tokyo", "month": "April"}}
🧩 Parsed Action text: {"name": "get_weather", "args": {"destination": "Tokyo", "month": "April"}}
👁️ Observation: The weather in Tokyo in April is cool and dry, highs ~15°C.
----------------------------------------------------------------------
Thought: I have the weather information. Next I should check the hotel prices to estimate costs.
Action: {"name": "get_hotel_prices", "args": {"destination": "Tokyo", "month": "April"}}
🧩 Parsed Action text: {"name": "get_hotel_prices", "args": {"destination": "Tokyo", "month": "April"}}
👁️ Observation: Hotels in Tokyo range from $100 to $300 per night in April.
----------------------------------------------------------------------
Thought: I have the weather and hotel price information. I should check the flight costs to complete the cost e